In [208]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from experiments.SL_bound import *
from experiments.DA_bound import *
from bounds.bounds import *
from util.kl import *
from util.misc import *
from util.batch import *
from results.plotting import *

# Hyper-parameters
task = 2
seed = 69105
n_classifiers = 10
delta=0.05 ## what would this be?   
binary=True
bound='germain'
epsilons=[0.01]
alphas=[0.1]#0,0.3]
sigmas=[[3,2],[3,3]]

project_folder = "/cephyr/users/frejohk/Alvis/projects/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Alvis params

In [112]:
project = 'SNIC2021-7-82'
username = 'frejohk'
job = 'batch_bound_single.sbatch'

In [113]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            103554     alvis batch_bo  frejohk PD       0:00      1 (Priority) 
            103465     alvis     bash  frejohk  R    3:25:15      1 alvis1-08 


In [114]:
!cat $job

#!/usr/bin/bash

#SBATCH -t 1-00:00:00
#SBATCH -N 1 --gpus-per-node=T4:1
#SBATCH -p alvis 

if [ -z "$task" ]
then
    task=2
fi
if [ -z "$seed" ]
then 
    seed=69105
fi
if [ -z "$alpha" ]
then 
    alpha=0.0
fi
if [ -z "$sigma" ]
then 
    sigma='3,3'
fi
if [ -z "$epsilon" ]
then 
    epsilon=0.01
fi
if [ -z "$delta" ]
then 
    delta=0.05
fi
if [ -z "$binary" ]
then 
    binary=0
fi
if [ -z "$nclassifiers" ]
then 
    nclassifiers=2
fi
if [ -z "$bound" ]
then 
    bound='germain'
fi
if [ -z "$prior" ]
then 
    prior=''
fi
if [ -z "$posterior" ]
then 
    posterior=''
fi

. load_modules.sh

echo $sigma

#python batch_bound_single.py -t $task -r $seed -a $alpha -s $sigma -e $epsilon -d $delta -b $binary -n $nclassifiers -B $bound -p $prior -P $posterior

In [202]:
print('Iterating over experiments...\n')

fids = []

os.makedirs('logs', exist_ok=True)


np.random.seed(seed)
for alpha in alphas:
    
    print("alpha:"+str(alpha))
    
    for epsilon in epsilons:
        print("  epsilon:"+str(epsilon))
        for sigma in sigmas:    
            print("    sigma:"+str(sigma))
            arg_list = get_job_args(task, bound=bound, alpha=alpha, sigma=sigma,
                                    epsilon=epsilon, binary=binary, n_classifiers=n_classifiers)
            for a in arg_list[5:]:   
                
                ckpt = os.path.splitext(os.path.basename(a['posterior_path']))[0]
                fid = './logs/batch_t-%d_r-%d_a-%.4f_s-%d%d_e-%.4f_d-%.4f_b-%d_n-%d_B-%s_c-%s' % \
                    (task, seed, alpha, sigma[0], sigma[1], epsilon, delta, binary, n_classifiers, bound, ckpt)
                
                sigstr = '"%d.%d"' % (sigma[0], sigma[1])
                exp = 'task=%d,seed=%d,alpha=%.4f,sigma="%s",epsilon=%.4f,delta=%.4f,'% (task, seed, alpha, sigstr, epsilon, delta)\
                        +'binary=%d,nclassifiers=%d,bound=%s,prior=%s,posterior=%s' % (binary, n_classifiers, bound, a['prior_path'], a['posterior_path'])
                
                prior_path = a['prior_path']
                output = !sbatch -o "$fid"-%j.out -e "$fid"-%j.err -A $project --export="$exp" $job

                jobid = int(output[0].split(' ')[-1])
                fid = fid+'-%s' % jobid

                fids.append(fid)

Iterating over experiments...

alpha:0.1
  epsilon:0.01
    sigma:[3, 2]
    sigma:[3, 3]


In [203]:
print(fids)

['./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_225-103639', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_270-103640', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_315-103641', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_360-103642', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_405-103643', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_450-103644', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_495-103645', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_540-103646', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_1-103647', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_2-103648', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germa

In [206]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            103465     alvis     bash  frejohk  R    3:55:21      1 alvis1-08 


In [207]:
for fid in fids: 
    print('------')
    print(fid)
    !tail -n 5 "$fid".out
    !tail -n 5 "$fid".err
    print(' \n\n')

------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_225-103639
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_1_225_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_270-103640
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_1_270_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_315-103641
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_1_315_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_360-103642
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_1_360_results.pkl ...
Done.
--------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_2-103648
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_2_2_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_3-103649
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_2_3_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_4-103650
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_2_4_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_5-103651
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_2_5_results.pkl ...
Done.
--------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_10-103656
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_2_10_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_11-103657
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_2_11_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_12-103658
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_2_12_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_13-103659
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_32_2_13_results.pkl ...
Done.
------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_405-103665
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_1_405_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_450-103666
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_1_450_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_495-103667
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_1_495_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-1_540-103668
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_1_540_results.pkl ...
Done.
----------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_5-103673
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_2_5_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_6-103674
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_2_6_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_7-103675
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_2_7_results.pkl ...
Done.
----------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_8-103676
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_2_8_results.pkl ...
Done.
--------------------------------------

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-2_B-germain_c-2_14-103682
Finished calculation of bound parts
Saving results in results/task2/Binary/10_10_33_2_14_results.pkl ...
Done.
----------------------------------------

 


